In [1]:
import os
import time
import shutil
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import keras
import gc
from PIL import Image
%matplotlib inline
from keras.models import Sequential,Model
from tensorflow.keras.applications import ResNet50, ResNet50V2, ResNet101V2
from keras.layers import Dense, Activation, Dropout
import keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D, Flatten, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
img_size = (512, 512)
batch_size = 8
# optimizer = SGD(learning_rate=5e-5, momentum=0.9)
optimizer = Adam(learning_rate=1e-4)
foldId = 0

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabels.csv")

indexes = []
for i in range(len(train_df['image'])):
    if train_df['kfold'][i] == foldId:
        indexes.append(i)
train_df.drop(indexes, axis=0, inplace=True)

train_df['image'] = [str(x)+'.jpeg' for x in train_df['image']]
train_df['level'] = [str(x) for x in train_df['level']]
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/train",
    x_col="image",
    y_col='level',
    subset="training",
    batch_size=8,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 25210 validated image filenames belonging to 5 classes.


In [4]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabels.csv")
indexes = []
for i in range(len(val_df['image'])):
    if val_df['kfold'][i] != foldId:
        indexes.append(i)
val_df.drop(indexes, axis=0, inplace=True)

val_df['image'] = [str(x)+'.jpeg' for x in val_df['image']]
val_df['level'] = [str(x) for x in val_df['level']]
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/train",
    x_col="image",
    y_col='level',
    batch_size=16,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 8404 validated image filenames belonging to 5 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/Data/Data/test_labels.csv")
test_df['image'] = [str(x)+'.jpeg' for x in test_df['image']]
test_df['level'] = [str(x) for x in test_df['level']]
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/test",
    x_col="image",
    y_col='level',
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=img_size)

Found 53574 validated image filenames belonging to 5 classes.


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [6]:
resnet = ResNet101V2(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))
output = resnet.layers[-1].output
output = keras.layers.Flatten()(output)
resnet = Model(resnet.input, output)

model = Sequential()
model.add(resnet)
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer, 
            loss='categorical_crossentropy',
            metrics=['accuracy'])

checkpoint_filepath = "./trainedmodel.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_filepath,
                          save_weights_only=True,
                          monitor='val_loss',
                          mode='auto',
                          save_best_only=True)

early_stopping = EarlyStopping(
                monitor="val_loss",
                patience=3,
                verbose=1,
                mode="auto",
)

model.summary()

2022-04-12 22:40:08.243221: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 22:40:08.327794: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 22:40:08.328615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 22:40:08.330204: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

171327488/171317808 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 524288)            42626560  
_________________________________________________________________
dense (Dense)                (None, 1024)              536871936 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 579,503,621
Trainable params: 579,405,957
Non-trainable params: 97,664
_________________________________________________________________


In [7]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [8]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20, 
                    callbacks=[model_checkpoint_callback, early_stopping])

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-04-12 22:40:15.459421: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-04-12 22:40:25.144487: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


3151/3151 [==============================] - 1522s 478ms/step - loss: 0.7864 - accuracy: 0.7667 - val_loss: 0.7446 - val_accuracy: 0.7618


2022-04-12 23:05:37.145715: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2147483648 exceeds 10% of free system memory.


...

Epoch 3/20
3151/3151 [==============================] - 1478s 469ms/step - loss: 0.5626 - accuracy: 0.8165 - val_loss: 0.6773 - val_accuracy: 0.7944


2022-04-12 23:55:15.289910: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2147483648 exceeds 10% of free system memory.


Epoch 4/20
3151/3151 [==============================] - 1462s 464ms/step - loss: 0.4978 - accuracy: 0.8391 - val_loss: 0.7746 - val_accuracy: 0.7950
Epoch 5/20
3151/3151 [==============================] - 1472s 467ms/step - loss: 0.4108 - accuracy: 0.8638 - val_loss: 0.6710 - val_accuracy: 0.8113


2022-04-13 00:45:18.084243: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2147483648 exceeds 10% of free system memory.


Epoch 6/20
3151/3151 [==============================] - 1470s 467ms/step - loss: 0.3150 - accuracy: 0.8911 - val_loss: 0.7274 - val_accuracy: 0.7927
Epoch 7/20
3151/3151 [==============================] - 1470s 466ms/step - loss: 0.2156 - accuracy: 0.9228 - val_loss: 1.1361 - val_accuracy: 0.8020
Epoch 8/20
3151/3151 [==============================] - 1472s 467ms/step - loss: 0.1530 - accuracy: 0.9465 - val_loss: 0.8911 - val_accuracy: 0.7985
Epoch 00008: early stopping


In [9]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=STEP_SIZE_TEST,
                            verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


53574/53574 [==============================] - 1618s 30ms/step


In [10]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames

res = {filenames[i]: predictions[i] for i in range(len(filenames))}

In [11]:
from collections import Counter
P = Counter(predictions)
T = Counter(test_df['level'])
print(sorted(P.items()))
print(sorted(T.items()))

[('0', 43287), ('1', 1724), ('2', 6801), ('3', 971), ('4', 791)]
[('0', 39533), ('1', 3762), ('2', 7861), ('3', 1214), ('4', 1206)]


In [12]:
correct = 0
for i in range(len(test_df['image'])):
    if test_df['image'][i] in res:
        if res[test_df['image'][i]] == test_df['level'][i]:
            correct += 1

acc = (correct / len(test_df['level'])) * 100
print(acc)

78.705017171868
